# Non-Human Primate (NHP) Motor Prediction Benchmark Tutorial

This tutorial aims to provide an insight on how the NeuroBench framework is organized and how you can use it to benchmark your own models!

## About NHP Motor Prediction:
Studying models which can accurately replicate features of biological computation presents opportunities in understanding sensorimotor behavior and developing closed-loop methods for future robotic agents. It also is foundational to the development of wearable or implantable neuro-prosthetic devices that can accurately generate motor activity from neural or muscle signals.

### Dataset:
The dataset that we utilize in this study consists of multi-channel recordings obtained from the sensorimotor cortex of two non-human primates (NHP) during self-paced reaching movements towards a grid of targets. A link to the dataset can be found [here](https://zenodo.org/records/583331). The variable $x$ is represented by threshold crossing times (or spike times) and sorted units for each of the recording channels. The target $y$ is represented by 2-dimensional position coordinates of the fingertip of the reaching hand, sampled at a frequency of 250 Hz. The complete dataset contains 37 sessions spanning 10 months for NHP-1 and 10 sessions from NHP-2 spanning one month. For this study, three sessions from each NHP were
selected to include the entire recording duration, resulting in a total of 8712 seconds of data.

### Benchmark Task:
In the context of predictive modeling, time series prediction is a task which entails the forecasting of one or more observations of a target variable, y, at some point between the current time, $t$, and a future time, $t$ + $t_f$ , by utilizing a sequence of another variable, $x$, from the past, {$x(t − th), . . . , x(t)$}. In the primate reaching task, the goal is to predict the $X$ and $Y$ components of finger velocity, $y$, from past neural data, $x$. The model may be trained separately for each session to account for inter-day neural variability. The data is split such that the first 75% of reaches are available for training and validation, while the latter 25% of reaches are used for evaluation.

First we will import the relevant libraries. These include the datasets, model wrapper, and benchmark object.

In [ ]:
import torch
from torch.utils.data import DataLoader, Subset

from neurobench.datasets import PrimateReaching
from neurobench.models.torch_model import TorchModel
from neurobench.benchmarks import Benchmark

For this tutorial, we demonstrate small ANN and SNN architectures. First, the ANN.

In [ ]:
from examples.primate_reaching.ANN import ANNModel2D

For this tutorial, we will only use one session from NHP Indy, `indy_20160622_01`. Note that there are 6 sessions in total, covering NHP Indy and NHP Loco. The dataset preprocesses the events into 4 ms bins, denoted by `bin_width`, such that the number of spikes for each of 96 cortical sensors during that time bin are given.

In [ ]:
filename = "indy_20160622_01"

# The dataloader and preprocessor has been combined together into a single class
data_dir = "../../../data/primate_reaching/PrimateReachingDataset/" # data in repo root dir
dataset = PrimateReaching(file_path=data_dir, filename=filename,
                        num_steps=1, train_ratio=0.5, bin_width=0.004,
                        biological_delay=0, remove_segments_inactive=False)

The data is treated such that each time series element is considered a separate sample. The sample is passed to the model, which predicts an X and Y velocity based on it and historical data. The data is shaped as `[num_samples, 1, 96]` for NHP Indy files (96 cortical sensors), and `[num_samples, 1, 192]` for NHP Loco files (192 cortical sensors), in order to fit with the three dimensional data format of NeuroBench.

In [ ]:
single_sample, single_label = dataset[0]
single_sample.shape

Now, wrap the dataset in a DataLoader. Importantly, the data should not be shuffled if the model expects that the data appears in temporal order. Since the ANN model uses a memory buffer to store recent data, shuffle must be False.

In [ ]:
test_set_loader = DataLoader(Subset(dataset, dataset.ind_test), batch_size=256, shuffle=False)

Instatiate then load the model. This is a pretrained feedforward linear model with two hidden layers, which uses a bin_window of 200 ms. Thus, it saves 50 timesteps in a memory buffer, since each timestep represents 4 ms. The spikes from the 50 most recent timesteps are summed to generate the input to the first layer of the model.

In [ ]:
net = ANNModel2D(input_dim=dataset.input_feature_size, layer1=32, layer2=48, 
                     output_dim=2, bin_window=0.2, drop_rate=0.5)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.load_state_dict(torch.load("./model_data/2D_ANN_Weight/"+filename+"_model_state_dict.pth", map_location=device))

model = TorchModel(net)

No pre- or post-processors are used for this task.

In [ ]:
preprocessors = []
postprocessors = []

Define the metrics. Correctness for this task is $R^2$, which is computed separately for the X and Y dimensions and averaged together.

In [ ]:
static_metrics = ["footprint", "connection_sparsity"]
workload_metrics = ["r2", "activation_sparsity", "synaptic_operations"]

Now you are ready to run the benchmark!


In [ ]:
# Benchmark expects the following:
benchmark = Benchmark(model, test_set_loader, [], [], [static_metrics, workload_metrics])
results = benchmark.run()
print(results)

Expected results for ANN:
{'footprint': 20824, 'connection_sparsity': 0.0, 'activation_sparsity': 0.7068512007122443, 'r2': 0.6327020525932312, 'synaptic_operations': {'Effective_MACs': 4306.322415210456, 'Effective_ACs': 0.0, 'Dense': 4702.261627687736}}

Now we will demonstrate the SNN benchmark. Import the model and related SNN units, and reinitialize the dataloader. Here, since the model is sequential, we are loading the entire dataset as a single batch.

In [ ]:
from examples.primate_reaching.SNN2 import SNN2
import snntorch as snn
test_set_loader = DataLoader(Subset(dataset, dataset.ind_test), batch_size=len(dataset.ind_test), shuffle=False)

Initialize and load the network. The SNN2 model architecture is a very simple two-layer linear feedforward SNN, which is implemented using SNNTorch.

In [ ]:
net = SNN2(input_size=dataset.input_feature_size)
net.load_state_dict(torch.load("./model_data/SNN2_{}.pt".format(filename), map_location=torch.device('cpu'))
                    ['model_state_dict'], strict=False)

Here, we wrap the model in a NeuroBenchModel wrapper. Notice that we use a TorchModel wrapper, rather than and SNNTorchModel wrapper, since the dataset is structured slightly differently. The SNNTorchModel wrapper expects that the timestep dimension of the data is given in dim 1, while the PrimateReaching dataset treats timesteps as samples and gives it in dim 0.

In [ ]:
# init the model
net.reset()
model = TorchModel(net) # using TorchModel instead of SNNTorchModel because the SNN iterates over dimension 0
model.add_activation_module(snn.SpikingNeuron)

The rest of the execution is the same as before! This should take about a minute.

In [ ]:
static_metrics = ["footprint", "connection_sparsity"]
workload_metrics = ["r2", "activation_sparsity", "synaptic_operations"]

# Benchmark expects the following:
benchmark = Benchmark(model, test_set_loader, [], [], [static_metrics, workload_metrics])
results = benchmark.run()
print(results)

Expected results for SNN:
{'footprint': 19648, 'connection_sparsity': 0.0, 'activation_sparsity': 0.9963387091440609, 'r2': 0.6774135828018188, 'synaptic_operations': {'Effective_MACs': 0.0, 'Effective_ACs': 396.6414365765915, 'Dense': 4900.0}}